# 🧩 Maze Scaling Benchmark - Proper Generation

**Using Research-Grade Maze Generation (maze-dataset library)**

This notebook uses the [maze-dataset](https://github.com/understanding-search/maze-dataset) library for high-quality maze generation with controllable difficulty.

## ✨ Key Improvements:
- ✅ **Proper algorithms**: DFS, Wilson's, Prim's (we use DFS for challenging mazes)
- ✅ **Path length filtering**: Control difficulty by minimum solution length
- ✅ **Guaranteed solvable**: No unsolvable mazes
- ✅ **Long planning horizons**: Force 80+ step solutions for differentiation

## Models Compared:
- **Baseline**: Standard Transformer encoder-decoder
- **PoH-HRM**: Pointer-over-Heads with Hierarchical Reasoning Module

## Why Proper Generation Matters:
**Old approach** (random walls): Path length ~16-20 → All models get 100% ❌  
**New approach** (DFS + filtering): Path length 80+ → Clear differentiation ✅

## Recommended Starting Point:
- **Maze Size**: 20×20 (large enough to be challenging)
- **Min Path Length**: 80 (20% of grid size)
- **Training**: 300-500 mazes
- **Expected**: Baseline: 60-75%, PoH-HRM: 80-90%

---

**Hardware**: Optimized for A100 GPU


## Setup


In [ ]:
# Clone repository
!git clone https://github.com/Eran-BA/PoT.git
%cd PoT


In [ ]:
# Install dependencies
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers datasets scipy numpy tqdm matplotlib seaborn

# Install maze-dataset library for proper maze generation
!pip install maze-dataset


In [ ]:
# Verify GPU
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
print(f"GPU Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB" if torch.cuda.is_available() else "N/A")


## Run Full Scaling Benchmark

**Default Configuration** (HRM Paper Protocol):
- Maze sizes: 8, 12, 16, 20, 24, 30
- 1,000 training mazes per size
- 200 test mazes per size
- 50 epochs per size
- PoH: R=4, T=4

**Expected Runtime on A100**: ~3-4 hours for full benchmark


In [ ]:
!python experiments/maze_scaling_benchmark.py \
  --maze-sizes 8 12 16 20 24 30 \
  --train 1000 \
  --test 200 \
  --R 4 \
  --T 4 \
  --heads 4 \
  --epochs 50 \
  --seed 42 \
  --output experiments/results/maze_scaling_full


## Quick Scaling Test (Faster)

**Reduced Configuration** for faster iteration:
- Maze sizes: 8, 12, 16, 20
- 500 training mazes per size
- 100 test mazes per size
- 30 epochs per size

**Expected Runtime on A100**: ~45-60 minutes


In [ ]:
!python experiments/maze_scaling_benchmark.py \
  --maze-sizes 8 12 16 20 \
  --train 500 \
  --test 100 \
  --R 4 \
  --T 4 \
  --heads 4 \
  --epochs 30 \
  --seed 42 \
  --output experiments/results/maze_scaling_quick


## Large-Scale Test (30×30 Only)

**Focus on hardest maze** (30×30) as in HRM paper:
- 1,000 training mazes
- 200 test mazes
- 100 epochs

**Expected Runtime on A100**: ~60-90 minutes


In [ ]:
!python experiments/maze_scaling_benchmark.py \
  --maze-sizes 30 \
  --train 1000 \
  --test 200 \
  --R 4 \
  --T 4 \
  --heads 4 \
  --epochs 100 \
  --seed 42 \
  --output experiments/results/maze_30x30_benchmark


## 🔥 HARDER MAZES (Better Model Differentiation)

**Problem**: Easy mazes lead to 100% accuracy for all models → no differentiation

**Solution**: Increase `wall_prob` to create harder mazes:
- `0.3` = Easy (default, often too easy)
- `0.45` = Medium
- `0.6` = Hard ⭐ **recommended for differentiation**
- `0.7` = Very Hard (may be unsolvable)

**Use the helper script** to easily run with different difficulties:


In [ ]:
# OPTION 1: Quick test with hard mazes (16×16 only, ~10 min)
!python experiments/run_harder_mazes.py --wall-prob 0.6 --quick

# OPTION 2: Full scaling with hard mazes (8-30, ~2 hours)
# !python experiments/run_harder_mazes.py --wall-prob 0.6 --maze-sizes 8 12 16 20 24 30

# OPTION 3: Medium difficulty full scaling
# !python experiments/run_harder_mazes.py --wall-prob 0.45 --maze-sizes 8 12 16 20 24 30

# OPTION 4: Very hard mazes (expert mode, may have no solution!)
# !python experiments/run_harder_mazes.py --wall-prob 0.7 --maze-sizes 8 12 16


## Visualize Results

The benchmark automatically generates:
1. **JSON results** with all metrics
2. **Scaling plot** showing accuracy/optimality vs maze size


In [ ]:
# Display the scaling plot
from IPython.display import Image, display

# Adjust path based on which benchmark you ran
display(Image('experiments/results/maze_scaling_full.png'))


In [ ]:
# Load and display JSON results
import json

with open('experiments/results/maze_scaling_full.json', 'r') as f:
    results = json.load(f)

print("Maze Scaling Results:")
print(json.dumps(results, indent=2))


## Analysis

**Key Questions:**
1. Does PoH-HRM outperform baselines on larger mazes (20×20+)?
2. How does performance degrade with maze size for each model?
3. Does HRM's hierarchical reasoning provide better path optimality?

**Expected Findings:**
- All models perform well on small mazes (8×8, 12×12)
- PoH-HRM should maintain high accuracy on 20×20+ mazes
- Baseline/BERT may struggle with longer planning horizons
- HRM's temporal abstraction (T=4) helps with multi-step reasoning


## Export Results

Download results to include in paper/documentation


In [ ]:
# Download results
from google.colab import files

files.download('experiments/results/maze_scaling_full.json')
files.download('experiments/results/maze_scaling_full.png')
